In [ ]:
from ltn_imp.automation.knowledge_base import KnowledgeBase

from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Prepare Datasets

In [ ]:
poker_hand = fetch_ucirepo(id=158)   
X = poker_hand.data.features 
y = poker_hand.data.targets 
data = pd.DataFrame(X, columns=poker_hand.data.feature_names)
data["Label"] = y

def prepare_datasets(data, random_seed=42):
    X = data.drop("Label", axis=1)  # Features
    y = data["Label"]  # Labels
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed, stratify=y)    
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=random_seed, stratify=y_train)
    
    train_data = pd.concat([X_train, y_train], axis=1)
    val_data = pd.concat([X_val, y_val], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)
    
    train_data.to_csv("datasets/train.csv", index=False)
    val_data.to_csv("datasets/val.csv", index=False)
    test_data.to_csv("datasets/test.csv", index=False)
    return train_data, val_data, test_data

# EDA 

In [ ]:
train = pd.read_csv("datasets/train.csv")
val = pd.read_csv("datasets/val.csv")
test = pd.read_csv("datasets/test.csv")
data = pd.concat([train, val, test])
print(train.shape)
print(val.shape)
print(test.shape)

In [ ]:
data["Label"].value_counts()

# Best Hyper-Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

def find_best_models(X,y):
    param_grid = {
        'KNN': {
            'model': KNeighborsClassifier(),
            'params': {
                'n_neighbors': [3, 5, 7],
                'weights': ['uniform', 'distance'],
                'p': [1, 2]
            }
        },
        'DT': {
            'model': DecisionTreeClassifier(),
            'params': {
                'max_depth': [None, 10, 20, 30],
                'min_samples_split': [2, 5, 10],
                'criterion': ['gini', 'entropy']
            }
        },
        'RF': {
            'model': RandomForestClassifier(),
            'params': {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5],
                'criterion': ['gini', 'entropy']
            }
        },
        'LR': {
            'model': LogisticRegression(max_iter=1000),
            'params': {
                'C': [0.1, 1, 10],
                'solver': ['lbfgs', 'liblinear']
            }
        }
    }

    best_models = {}
    for name, model_info in param_grid.items():
        grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=5, n_jobs=-1)
        grid_search.fit(X, y)
        best_models[name] = grid_search.best_estimator_

    models = []
    for name, model in best_models.items():
        if name == 'KNN':
            models.append(KNeighborsClassifier(**model.get_params()))
        elif name == 'DT':
            models.append(DecisionTreeClassifier(**model.get_params()))
        elif name == 'RF':
            models.append(RandomForestClassifier(**model.get_params()))
        elif name == 'LR':
            models.append(LogisticRegression(**model.get_params()))

    return models

In [ ]:
models = find_best_models(train.drop("Label", axis=1), train["Label"])

In [ ]:
kb = KnowledgeBase("config.yaml")

In [ ]:
models.append(kb.predicates["PokerHand"])

# Evaluation

In [ ]:
def evaluate_model(loader, model, device):
    all_labels = []
    all_predictions = []

    # Iterate over the data loader
    for data, labels in loader:
        # Move data and labels to the specified device
        data = data.to(device)
        labels = labels.to(device)

        # Get predictions from the model
        if isinstance(model, torch.nn.Module):            
            with torch.no_grad():
                predictions = model(data)
                predicted_labels = torch.argmax(predictions, dim=1)
        else:
            predicted_labels = model.predict(data)

    if isinstance(model, torch.nn.Module):
        all_labels.extend(labels.cpu().numpy())  
        all_predictions.extend(predicted_labels.cpu().numpy()) 
    else:
        all_labels.extend(labels)
        all_predictions.extend(predicted_labels)

    if isinstance(model, torch.nn.Module):
        all_labels = np.array(all_labels)
        all_predictions = np.array(all_predictions)

    # Compute sklearn metrics
    overall_accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro') 
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')

    return overall_accuracy, precision, recall, f1

In [ ]:
def plot_metrics(metrics_df, metrics_to_plot):
    plt.figure(figsize=(18, len(metrics_to_plot) * 4))

    for i, metric in enumerate(metrics_to_plot, 1):
        plt.subplot(len(metrics_to_plot), 1, i)
        data_to_plot = [metrics_df.loc[model_name, metric] for model_name in metrics_df.index]
        
        # Customize the boxplots
        boxprops = dict(linewidth=2)
        medianprops = dict(linewidth=2, color='red')
        meanprops = dict(linewidth=2, color='blue')
        whiskerprops = dict(linewidth=2)
        capprops = dict(linewidth=2)
        
        plt.boxplot(data_to_plot, labels=metrics_df.index, boxprops=boxprops, 
                    medianprops=medianprops, meanline=True, showmeans=True, 
                    meanprops=meanprops, whiskerprops=whiskerprops, 
                    capprops=capprops)
        
        plt.title(f'Boxplot of {metric} across different models')
        plt.xlabel('Model', fontweight='bold')
        plt.ylabel(metric, fontweight='bold')

    plt.tight_layout()
    plt.show()

In [ ]:
import scipy.stats as stats

def compare_models(metrics_df):

    best_models = {}
    for metric in metrics_df.columns:
        try:
            mean_scores = metrics_df[metric].apply(np.mean)
            best_model = mean_scores.idxmax()  
            best_models[metric] = best_model
        except:
            print(f"Rule {metric} not available")
            continue

    results = {}
    for metric in metrics_df.columns:
        try:   
            best_model = best_models[metric]
            best_scores = metrics_df.loc[best_model, metric]
            
            results[metric] = {}
            
            for model_name in metrics_df.index:
                if model_name == best_model:
                    continue
                
                comparison_scores = metrics_df.loc[model_name, metric]
                t_stat, p_value = stats.ttest_rel(best_scores, comparison_scores)
                results[metric][model_name] = p_value 

        except:
            print(f"Rule {metric} not available")
            continue

    significance_level = 0.05
    for metric, comparisons in results.items():
        try:
            print(f"\n{metric}:")
            best_model = best_models[metric]
            for model_name, p_value in comparisons.items():
                if p_value < significance_level:
                    print(f"  {best_model} is significantly better than {model_name} (p = {p_value:.4f})")
                else:
                    print(f"  {best_model} is NOT significantly better than {model_name} (p = {p_value:.4f})")
        except:
            print(f"Rule {metric} not available")

In [ ]:
def perform_t_tests(metrics_df, model_name):
    results = {}

    for metric in metrics_df.columns:

        if model_name == "SKI MLP":
            continue
        
        # Retrieve the scores for SKI MLP and Regular MLP
        ski_mlp_scores = np.array(metrics_df.loc['SKI MLP', metric][0])
        regular_mlp_scores = np.array(metrics_df.loc[model_name, metric][0])
        
        # Perform a paired t-test between SKI MLP and Regular MLP
        t_stat, p_value = stats.ttest_rel(ski_mlp_scores, regular_mlp_scores)
        
        # Calculate the mean difference
        mean_difference = np.mean(ski_mlp_scores - regular_mlp_scores)
        
        results[metric] = {'p_value': p_value, 'mean_difference': mean_difference}

    # Significance level for the tests
    significance_level = 0.05

    for metric, result in results.items():
        try:
            p_value = result['p_value']
            mean_difference = result['mean_difference']
            
            print(f"\n{metric}:")
            if p_value < significance_level:
                if mean_difference > 0:
                    print(f"  SKI MLP is significantly better than {model_name} (p = {p_value:.4f})")
                else:
                    print(f"  {model_name} is significantly better than SKI MLP (p = {p_value:.4f})")
            else:
                print(f"  There is no significant difference between SKI MLP and {model_name} (p = {p_value:.4f})")
        except:
            print(f"Rule {metric} not available")

In [ ]:
def train_model(model, train_loader, val_loader, device, patience=5, max_epochs=100):
    # Set the model to training mode
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=0.0001, weight_decay=0.0001)

    # Early stopping variables
    best_val_loss = np.inf
    epochs_no_improve = 0

    # Loop over epochs
    for _ in range(max_epochs):
        model.train()
        running_loss = 0.0

        # Training loop
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            labels = labels.view(-1).long()
            
            optimizer.zero_grad()
            outputs = model(data)            
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for data, labels in val_loader:
                data, labels = data.to(device), labels.to(device)
                labels = labels.view(-1).long()
                outputs = model(data)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0  # Reset counter if validation loss improves
            best_model = model.state_dict()  # Save the best model
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            break

    model.load_state_dict(best_model)
    return model

In [ ]:
model_names = [type(model).__name__ for model in models]
model_names.remove("Sequential")
model_names.append("Regular MLP")
model_names.append("SKI MLP")
metrics = ["Overall Accuracy", "Precision", "Recall", "F1"]
metrics_df = pd.DataFrame([ [ [] for _ in metrics ] for _ in model_names ] , columns=metrics, index=[model_names])

seeds = [random.randint(0, 1000) for _ in range(2)]

for seed in seeds:
    train, val, test = prepare_datasets(data, seed)
    for model in models:
        model_name = type(model).__name__
        if isinstance(model, torch.nn.Module):
            kb = KnowledgeBase("config.yaml")
            model = kb.predicates["PokerHand"]

            # Before Fine Tuning With Logic 
            train_model(model, kb.loaders[0], kb.val_loaders[0], kb.device)
            metrics_values = evaluate_model(kb.test_loaders[0], model, kb.device)
            for metric, value in zip(metrics, metrics_values):
                metrics_df.loc["Regular MLP"][metric][0].append(value)
            
            # After Fine Tuning With Logic 
            kb.optimize(num_epochs=200, lr=0.000001, early_stopping=True, verbose=False)
            model_name = "SKI MLP"
        else:
            model.fit(train.drop("Label", axis=1), train["Label"])

        metrics_values = evaluate_model(kb.test_loaders[0], model, kb.device)
        for metric, value in zip(metrics, metrics_values):
            metrics_df.loc[model_name][metric][0].append(value)

In [ ]:
plot_metrics(metrics_df, metrics)

In [ ]:
compare_models(metrics_df)

In [ ]:
for model in model_names:
    perform_t_tests(metrics_df, model)
    print()
    print( "-" * 100)